In [1]:
import streamlit as st
from langchain.agents import Tool
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback
from langchain.embeddings import OpenAIEmbeddings
from langchain_experimental.autonomous_agents import AutoGPT
from langchain.tools.file_management.write import WriteFileTool
from langchain.tools.file_management.read import ReadFileTool
from langchain.utilities import SerpAPIWrapper

In [2]:
openai_api_key = st.secrets['openai_api_key']
serpapi_api_key = st.secrets['serpapi_api_key']
model_name = 'gpt-3.5-turbo'

In [9]:



def load_vectordb():
    import faiss
    from langchain.vectorstores import FAISS
    from langchain.docstore import InMemoryDocstore
    
    embedding_size = 1536
    embedding = OpenAIEmbeddings(openai_api_key=openai_api_key)
    # 使用 faiss 的 IndexFlatL2 索引
    index = faiss.IndexFlatL2(embedding_size)
    # 实例化 Faiss 向量数据库
    vectordb = FAISS(embedding.embed_query, index, InMemoryDocstore({}), {})
    return vectordb

def load_tools():
    search = SerpAPIWrapper(serpapi_api_key=serpapi_api_key)
    tools = [
        Tool(
            name="search",
            func=search.run,
            description="useful for when you need to answer questions about current events. You should ask targeted questions",
        ),
        WriteFileTool(),
        ReadFileTool(),
    ]
    return tools


def load_llm():
    chat = ChatOpenAI(openai_api_key=openai_api_key, temperature=0, model_name=model_name)
    return chat


def load_agent(llm, tools, vectordb):
    agent = AutoGPT.from_llm_and_tools(
        ai_name="Xiaoming",
        ai_role="Assistant",
        tools=tools,
        llm=llm,
        memory=vectordb.as_retriever()
    )
    agent.chain.verbose = True
    return agent


def run(agent, question):
    with get_openai_callback() as cb:
        agent.run([question])
        print(cb)

In [8]:
vectordb = load_vectordb()
agent = load_agent(load_llm(), load_tools(), vectordb)
    

In [10]:
question = '今年亚运会在哪里举行'
agent.run([question])



> Entering new LLMChain chain...
Prompt after formatting:
System: You are Xiaoming, Assistant
Your decisions must always be made independently without seeking user assistance.
Play to your strengths as an LLM and pursue simple strategies with no legal complications.
If you have completed all your tasks, make sure to use the "finish" command.

GOALS:

1. 今年亚运会在哪里举行


Constraints:
1. ~4000 word limit for short term memory. Your short term memory is short, so immediately save important information to files.
2. If you are unsure how you previously did something or want to recall past events, thinking about similar events will help you remember.
3. No user assistance
4. Exclusively use the commands listed in double quotes e.g. "command name"

Commands:
1. search: useful for when you need to answer questions about current events. You should ask targeted questions, args json schema: {"tool_input": {"type": "string"}}
2. write_file: Write file to disk, args json schema: {"file_path": {"title

'The 19th Asian Games will be held in Hangzhou, China.'

In [11]:
query = question
result = vectordb.similarity_search(query)

In [15]:
result[0].page_content


'Assistant Reply: {\n    "thoughts": {\n        "text": "I need to determine which next command to use.",\n        "reasoning": "To determine the next command, I should assess the current situation and identify the task at hand. Once I know what needs to be done, I can choose the appropriate command to accomplish the task.",\n        "plan": "- Assess the current situation\\n- Identify the task at hand\\n- Choose the appropriate command",\n        "criticism": "None",\n        "speak": "I am currently assessing the situation to determine the next command."\n    },\n    "command": {\n        "name": "search",\n        "args": {\n            "tool_input": "亚运会 举办地点"\n        }\n    }\n} \nResult: Command search returned: [\'The Asian Games, also known as Asiad, is a continental multi-sport event held every fourth year among athletes from all over Asia. The Games were regulated by the Asian Games Federation from the first Games in New Delhi, India, until the 1978 Games.\', \'Asian Games (